In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

from os.path import join

plt.style.use(["seaborn", "thesis"])

/home/jcartus/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Data

In [2]:
from SCFInitialGuess.utilities.dataset import extract_triu_batch, AbstractDataset
from sklearn.model_selection import train_test_split

#data_path = "../dataset/TSmall"
#postfix = "TSmall"
#dim = 70
data_path = "../../dataset/TSmall_sto3g"
postfix = "TSmall_sto3g"
dim = 26
#data_path = "../butadien/data/"
#postfix = ""
#dim = 26


def split(x, y, ind):
    return x[:ind], y[:ind], x[ind:], y[ind:]

S = np.load(join(data_path, "S" + postfix + ".npy"))
P = np.load(join(data_path, "P" + postfix + ".npy"))
F = np.load(join(data_path, "F" + postfix + ".npy"))

index = np.load(join(data_path, "index" + postfix + ".npy"))

molecules = np.load(join(data_path, "molecules" + postfix + ".npy"))



ind = int(0.8 * len(index))

s_triu = extract_triu_batch(S, dim)
p_triu = extract_triu_batch(P, dim)

s_triu_norm, mu, std = AbstractDataset.normalize(s_triu)


s_train, p_train, s_test, p_test = split(s_triu_norm, p_triu, ind)

# Model

In [3]:
filepath = "../../models/" + "TSmall_sto3gmodelTSmall_sto3g_250-150-50+triu.h5"
model = keras.models.load_model(filepath)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 351)               123552    
_________________________________________________________________
dense_1 (Dense)              (None, 601)               211552    
_________________________________________________________________
dense_2 (Dense)              (None, 501)               301602    
_________________________________________________________________
dense_3 (Dense)              (None, 401)               201302    
_________________________________________________________________
dense_4 (Dense)              (None, 351)               141102    
Total params: 979,110
Trainable params: 979,110
Non-trainable params: 0
_________________________________________________________________


# Calculate Guesses

In [10]:
from SCFInitialGuess.utilities.dataset import make_matrix_batch
from SCFInitialGuess.nn.post_processing import multi_mc_wheeny
p_nn = model.predict(s_test)

s_raw = make_matrix_batch(s_triu[ind:], dim, is_triu=True)

p_batch = make_matrix_batch(p_nn, dim, True).astype("float64")
p_mcw1 = np.array(list(map(lambda x: multi_mc_wheeny(x[0], x[1], n_max=1), zip(p_batch, s_raw)))).astype("float64")
p_mcw5 = np.array(list(map(lambda x: multi_mc_wheeny(x[0], x[1], n_max=5), zip(p_batch, s_raw)))).astype("float64")

# Analysis Prerequisits

In [11]:
from SCFInitialGuess.utilities.analysis import make_results_str, measure_all_quantities

N_ELECTRONS = 30


In [13]:
from SCFInitialGuess.utilities.dataset import StaticDataset

dataset = StaticDataset(
    train=(s_train, p_train),
    validation=(None, None),
    test=(s_test, p_test),
    mu=mu,
    std=std
)

In [15]:
def format_results(result):
    if isinstance(result, list):
        out = list(map(
            lambda x: "{:0.5E} +- {:0.5E}".format(*x),
            result
        ))
        out = "\n".join(out)
    else:
        out =  "{:0.5E} +- {:0.5E}".format(*result)
    return out

# Analysis McW1

In [18]:
from SCFInitialGuess.utilities.analysis import mf_initializer as mf_initializer

print(make_results_str(measure_all_quantities(
    p_mcw1,
    dataset,
    molecules[ind:],
    N_ELECTRONS,
    mf_initializer,
    dim,
    is_triu=False,
    is_dataset_triu=True,
    s=S[ind:]
)))

[ ] 2018-10-03 13:33:17: Iteration calculation: 0
[ ] 2018-10-03 13:33:18: Iteration calculation: 1
[ ] 2018-10-03 13:33:18: Iteration calculation: 2
[ ] 2018-10-03 13:33:19: Iteration calculation: 3
[ ] 2018-10-03 13:33:19: Iteration calculation: 4
[ ] 2018-10-03 13:33:19: Iteration calculation: 5
[ ] 2018-10-03 13:33:20: Iteration calculation: 6
[ ] 2018-10-03 13:33:20: Iteration calculation: 7
[ ] 2018-10-03 13:33:20: Iteration calculation: 8
[ ] 2018-10-03 13:33:21: Iteration calculation: 9
[ ] 2018-10-03 13:33:21: Iteration calculation: 10
[ ] 2018-10-03 13:33:21: Iteration calculation: 11
[ ] 2018-10-03 13:33:22: Iteration calculation: 12
[ ] 2018-10-03 13:33:23: Iteration calculation: 13
[ ] 2018-10-03 13:33:24: Iteration calculation: 14
[ ] 2018-10-03 13:33:24: Iteration calculation: 15
[ ] 2018-10-03 13:33:25: Iteration calculation: 16
[ ] 2018-10-03 13:33:25: Iteration calculation: 17
[ ] 2018-10-03 13:33:25: Iteration calculation: 18
[ ] 2018-10-03 13:33:26: Iteration calcul

[ ] 2018-10-03 13:34:17: Iteration calculation: 160
[ ] 2018-10-03 13:34:17: Iteration calculation: 161
[ ] 2018-10-03 13:34:17: Iteration calculation: 162
[ ] 2018-10-03 13:34:17: Iteration calculation: 163
[ ] 2018-10-03 13:34:18: Iteration calculation: 164
[ ] 2018-10-03 13:34:18: Iteration calculation: 165
[ ] 2018-10-03 13:34:18: Iteration calculation: 166
[ ] 2018-10-03 13:34:18: Iteration calculation: 167
[ ] 2018-10-03 13:34:19: Iteration calculation: 168
[ ] 2018-10-03 13:34:19: Iteration calculation: 169
[ ] 2018-10-03 13:34:19: Iteration calculation: 170
[ ] 2018-10-03 13:34:20: Iteration calculation: 171
[ ] 2018-10-03 13:34:20: Iteration calculation: 172
[ ] 2018-10-03 13:34:20: Iteration calculation: 173
[ ] 2018-10-03 13:34:20: Iteration calculation: 174
[ ] 2018-10-03 13:34:21: Iteration calculation: 175
[ ] 2018-10-03 13:34:21: Iteration calculation: 176
[ ] 2018-10-03 13:34:21: Iteration calculation: 177
[ ] 2018-10-03 13:34:22: Iteration calculation: 178
[ ] 2018-10-

In [20]:
from SCFInitialGuess.utilities.analysis import mf_initializer_damping, measure_iterations, statistics

print("--- Iterations Damped ---\n" + \
format_results(statistics(list(measure_iterations(
    mf_initializer_damping,
   p_mcw1.astype('float64'),
    molecules[ind:]
)))))

[ ] 2018-10-03 13:35:25: Iteration calculation: 0
[ ] 2018-10-03 13:35:25: Iteration calculation: 1
[ ] 2018-10-03 13:35:25: Iteration calculation: 2
[ ] 2018-10-03 13:35:26: Iteration calculation: 3
[ ] 2018-10-03 13:35:26: Iteration calculation: 4
[ ] 2018-10-03 13:35:26: Iteration calculation: 5
[ ] 2018-10-03 13:35:27: Iteration calculation: 6
[ ] 2018-10-03 13:35:27: Iteration calculation: 7
[ ] 2018-10-03 13:35:27: Iteration calculation: 8
[ ] 2018-10-03 13:35:27: Iteration calculation: 9
[ ] 2018-10-03 13:35:28: Iteration calculation: 10
[ ] 2018-10-03 13:35:28: Iteration calculation: 11
[ ] 2018-10-03 13:35:28: Iteration calculation: 12
[ ] 2018-10-03 13:35:29: Iteration calculation: 13
[ ] 2018-10-03 13:35:29: Iteration calculation: 14
[ ] 2018-10-03 13:35:29: Iteration calculation: 15
[ ] 2018-10-03 13:35:29: Iteration calculation: 16
[ ] 2018-10-03 13:35:30: Iteration calculation: 17
[ ] 2018-10-03 13:35:30: Iteration calculation: 18
[ ] 2018-10-03 13:35:30: Iteration calcul

[ ] 2018-10-03 13:36:12: Iteration calculation: 160
[ ] 2018-10-03 13:36:12: Iteration calculation: 161
[ ] 2018-10-03 13:36:13: Iteration calculation: 162
[ ] 2018-10-03 13:36:13: Iteration calculation: 163
[ ] 2018-10-03 13:36:13: Iteration calculation: 164
[ ] 2018-10-03 13:36:14: Iteration calculation: 165
[ ] 2018-10-03 13:36:14: Iteration calculation: 166
[ ] 2018-10-03 13:36:14: Iteration calculation: 167
[ ] 2018-10-03 13:36:14: Iteration calculation: 168
[ ] 2018-10-03 13:36:15: Iteration calculation: 169
[ ] 2018-10-03 13:36:15: Iteration calculation: 170
[ ] 2018-10-03 13:36:15: Iteration calculation: 171
[ ] 2018-10-03 13:36:15: Iteration calculation: 172
[ ] 2018-10-03 13:36:16: Iteration calculation: 173
[ ] 2018-10-03 13:36:16: Iteration calculation: 174
[ ] 2018-10-03 13:36:16: Iteration calculation: 175
[ ] 2018-10-03 13:36:16: Iteration calculation: 176
[ ] 2018-10-03 13:36:17: Iteration calculation: 177
[ ] 2018-10-03 13:36:17: Iteration calculation: 178
[ ] 2018-10-

In [21]:
from SCFInitialGuess.utilities.analysis import mf_initializer_diis, measure_iterations, statistics

print("--- Iterations DIIS ---\n" + \
format_results(statistics(list(measure_iterations(
    mf_initializer_diis,
    p_mcw1.astype('float64'),
    molecules[ind:]
)))))

[ ] 2018-10-03 13:36:24: Iteration calculation: 0
[ ] 2018-10-03 13:36:24: Iteration calculation: 1
[ ] 2018-10-03 13:36:24: Iteration calculation: 2
[ ] 2018-10-03 13:36:25: Iteration calculation: 3
[ ] 2018-10-03 13:36:25: Iteration calculation: 4
[ ] 2018-10-03 13:36:25: Iteration calculation: 5
[ ] 2018-10-03 13:36:26: Iteration calculation: 6
[ ] 2018-10-03 13:36:26: Iteration calculation: 7
[ ] 2018-10-03 13:36:26: Iteration calculation: 8
[ ] 2018-10-03 13:36:27: Iteration calculation: 9
[ ] 2018-10-03 13:36:27: Iteration calculation: 10
[ ] 2018-10-03 13:36:27: Iteration calculation: 11
[ ] 2018-10-03 13:36:28: Iteration calculation: 12
[ ] 2018-10-03 13:36:28: Iteration calculation: 13
[ ] 2018-10-03 13:36:28: Iteration calculation: 14
[ ] 2018-10-03 13:36:29: Iteration calculation: 15
[ ] 2018-10-03 13:36:29: Iteration calculation: 16
[ ] 2018-10-03 13:36:29: Iteration calculation: 17
[ ] 2018-10-03 13:36:30: Iteration calculation: 18
[ ] 2018-10-03 13:36:30: Iteration calcul

[ ] 2018-10-03 13:37:10: Iteration calculation: 161
[ ] 2018-10-03 13:37:10: Iteration calculation: 162
[ ] 2018-10-03 13:37:10: Iteration calculation: 163
[ ] 2018-10-03 13:37:11: Iteration calculation: 164
[ ] 2018-10-03 13:37:11: Iteration calculation: 165
[ ] 2018-10-03 13:37:11: Iteration calculation: 166
[ ] 2018-10-03 13:37:11: Iteration calculation: 167
[ ] 2018-10-03 13:37:12: Iteration calculation: 168
[ ] 2018-10-03 13:37:12: Iteration calculation: 169
[ ] 2018-10-03 13:37:12: Iteration calculation: 170
[ ] 2018-10-03 13:37:13: Iteration calculation: 171
[ ] 2018-10-03 13:37:13: Iteration calculation: 172
[ ] 2018-10-03 13:37:13: Iteration calculation: 173
[ ] 2018-10-03 13:37:14: Iteration calculation: 174
[ ] 2018-10-03 13:37:14: Iteration calculation: 175
[ ] 2018-10-03 13:37:14: Iteration calculation: 176
[ ] 2018-10-03 13:37:14: Iteration calculation: 177
[ ] 2018-10-03 13:37:15: Iteration calculation: 178
[ ] 2018-10-03 13:37:15: Iteration calculation: 179
[ ] 2018-10-

# McW5 

In [22]:
from SCFInitialGuess.utilities.analysis import mf_initializer as mf_initializer

print(make_results_str(measure_all_quantities(
    p_mcw5,
    dataset,
    molecules[ind:],
    N_ELECTRONS,
    mf_initializer,
    dim,
    is_triu=False,
    is_dataset_triu=True,
    s=S[ind:]
)))

[ ] 2018-10-03 13:50:40: Iteration calculation: 0
[ ] 2018-10-03 13:50:41: Iteration calculation: 1
[ ] 2018-10-03 13:50:41: Iteration calculation: 2
[ ] 2018-10-03 13:50:42: Iteration calculation: 3
[ ] 2018-10-03 13:50:43: Iteration calculation: 4
[ ] 2018-10-03 13:50:43: Iteration calculation: 5
[ ] 2018-10-03 13:50:43: Iteration calculation: 6
[ ] 2018-10-03 13:50:44: Iteration calculation: 7
[ ] 2018-10-03 13:50:44: Iteration calculation: 8
[ ] 2018-10-03 13:50:44: Iteration calculation: 9
[ ] 2018-10-03 13:50:45: Iteration calculation: 10
[ ] 2018-10-03 13:50:45: Iteration calculation: 11
[ ] 2018-10-03 13:50:45: Iteration calculation: 12
[ ] 2018-10-03 13:50:47: Iteration calculation: 13
[ ] 2018-10-03 13:50:48: Iteration calculation: 14
[ ] 2018-10-03 13:50:48: Iteration calculation: 15
[ ] 2018-10-03 13:50:48: Iteration calculation: 16
[ ] 2018-10-03 13:50:49: Iteration calculation: 17
[ ] 2018-10-03 13:50:49: Iteration calculation: 18
[ ] 2018-10-03 13:50:49: Iteration calcul

[ ] 2018-10-03 13:51:37: Iteration calculation: 160
[ ] 2018-10-03 13:51:37: Iteration calculation: 161
[ ] 2018-10-03 13:51:38: Iteration calculation: 162
[ ] 2018-10-03 13:51:38: Iteration calculation: 163
[ ] 2018-10-03 13:51:38: Iteration calculation: 164
[ ] 2018-10-03 13:51:39: Iteration calculation: 165
[ ] 2018-10-03 13:51:39: Iteration calculation: 166
[ ] 2018-10-03 13:51:39: Iteration calculation: 167
[ ] 2018-10-03 13:51:40: Iteration calculation: 168
[ ] 2018-10-03 13:51:40: Iteration calculation: 169
[ ] 2018-10-03 13:51:40: Iteration calculation: 170
[ ] 2018-10-03 13:51:41: Iteration calculation: 171
[ ] 2018-10-03 13:51:41: Iteration calculation: 172
[ ] 2018-10-03 13:51:41: Iteration calculation: 173
[ ] 2018-10-03 13:51:41: Iteration calculation: 174
[ ] 2018-10-03 13:51:42: Iteration calculation: 175
[ ] 2018-10-03 13:51:42: Iteration calculation: 176
[ ] 2018-10-03 13:51:43: Iteration calculation: 177
[ ] 2018-10-03 13:51:43: Iteration calculation: 178
[ ] 2018-10-

In [24]:
from SCFInitialGuess.utilities.analysis import mf_initializer_damping, measure_iterations, statistics

print("--- Iterations DAMPED ---\n" + \
format_results(statistics(list(measure_iterations(
    mf_initializer_damping,
    p_mcw5.astype('float64'),
    molecules[ind:]
)))))

[ ] 2018-10-03 13:54:14: Iteration calculation: 0
[ ] 2018-10-03 13:54:14: Iteration calculation: 1
[ ] 2018-10-03 13:54:14: Iteration calculation: 2
[ ] 2018-10-03 13:54:14: Iteration calculation: 3
[ ] 2018-10-03 13:54:15: Iteration calculation: 4
[ ] 2018-10-03 13:54:15: Iteration calculation: 5
[ ] 2018-10-03 13:54:15: Iteration calculation: 6
[ ] 2018-10-03 13:54:15: Iteration calculation: 7
[ ] 2018-10-03 13:54:16: Iteration calculation: 8
[ ] 2018-10-03 13:54:16: Iteration calculation: 9
[ ] 2018-10-03 13:54:16: Iteration calculation: 10
[ ] 2018-10-03 13:54:17: Iteration calculation: 11
[ ] 2018-10-03 13:54:17: Iteration calculation: 12
[ ] 2018-10-03 13:54:17: Iteration calculation: 13
[ ] 2018-10-03 13:54:18: Iteration calculation: 14
[ ] 2018-10-03 13:54:18: Iteration calculation: 15
[ ] 2018-10-03 13:54:18: Iteration calculation: 16
[ ] 2018-10-03 13:54:19: Iteration calculation: 17
[ ] 2018-10-03 13:54:19: Iteration calculation: 18
[ ] 2018-10-03 13:54:19: Iteration calcul

[ ] 2018-10-03 13:55:02: Iteration calculation: 161
[ ] 2018-10-03 13:55:02: Iteration calculation: 162
[ ] 2018-10-03 13:55:02: Iteration calculation: 163
[ ] 2018-10-03 13:55:03: Iteration calculation: 164
[ ] 2018-10-03 13:55:03: Iteration calculation: 165
[ ] 2018-10-03 13:55:03: Iteration calculation: 166
[ ] 2018-10-03 13:55:03: Iteration calculation: 167
[ ] 2018-10-03 13:55:04: Iteration calculation: 168
[ ] 2018-10-03 13:55:04: Iteration calculation: 169
[ ] 2018-10-03 13:55:04: Iteration calculation: 170
[ ] 2018-10-03 13:55:04: Iteration calculation: 171
[ ] 2018-10-03 13:55:05: Iteration calculation: 172
[ ] 2018-10-03 13:55:05: Iteration calculation: 173
[ ] 2018-10-03 13:55:05: Iteration calculation: 174
[ ] 2018-10-03 13:55:05: Iteration calculation: 175
[ ] 2018-10-03 13:55:06: Iteration calculation: 176
[ ] 2018-10-03 13:55:06: Iteration calculation: 177
[ ] 2018-10-03 13:55:06: Iteration calculation: 178
[ ] 2018-10-03 13:55:06: Iteration calculation: 179
[ ] 2018-10-

In [25]:
from SCFInitialGuess.utilities.analysis import mf_initializer_diis, measure_iterations, statistics

print("--- Iterations DIIS ---\n" + \
format_results(statistics(list(measure_iterations(
    mf_initializer_diis,
    p_mcw5.astype('float64'),
    molecules[ind:]
)))))

[ ] 2018-10-03 13:55:12: Iteration calculation: 0
[ ] 2018-10-03 13:55:12: Iteration calculation: 1
[ ] 2018-10-03 13:55:13: Iteration calculation: 2
[ ] 2018-10-03 13:55:13: Iteration calculation: 3
[ ] 2018-10-03 13:55:14: Iteration calculation: 4
[ ] 2018-10-03 13:55:14: Iteration calculation: 5
[ ] 2018-10-03 13:55:14: Iteration calculation: 6
[ ] 2018-10-03 13:55:14: Iteration calculation: 7
[ ] 2018-10-03 13:55:15: Iteration calculation: 8
[ ] 2018-10-03 13:55:15: Iteration calculation: 9
[ ] 2018-10-03 13:55:15: Iteration calculation: 10
[ ] 2018-10-03 13:55:15: Iteration calculation: 11
[ ] 2018-10-03 13:55:16: Iteration calculation: 12
[ ] 2018-10-03 13:55:16: Iteration calculation: 13
[ ] 2018-10-03 13:55:16: Iteration calculation: 14
[ ] 2018-10-03 13:55:16: Iteration calculation: 15
[ ] 2018-10-03 13:55:17: Iteration calculation: 16
[ ] 2018-10-03 13:55:17: Iteration calculation: 17
[ ] 2018-10-03 13:55:17: Iteration calculation: 18
[ ] 2018-10-03 13:55:17: Iteration calcul

[ ] 2018-10-03 13:55:51: Iteration calculation: 160
[ ] 2018-10-03 13:55:51: Iteration calculation: 161
[ ] 2018-10-03 13:55:51: Iteration calculation: 162
[ ] 2018-10-03 13:55:52: Iteration calculation: 163
[ ] 2018-10-03 13:55:52: Iteration calculation: 164
[ ] 2018-10-03 13:55:52: Iteration calculation: 165
[ ] 2018-10-03 13:55:52: Iteration calculation: 166
[ ] 2018-10-03 13:55:53: Iteration calculation: 167
[ ] 2018-10-03 13:55:53: Iteration calculation: 168
[ ] 2018-10-03 13:55:53: Iteration calculation: 169
[ ] 2018-10-03 13:55:53: Iteration calculation: 170
[ ] 2018-10-03 13:55:54: Iteration calculation: 171
[ ] 2018-10-03 13:55:54: Iteration calculation: 172
[ ] 2018-10-03 13:55:54: Iteration calculation: 173
[ ] 2018-10-03 13:55:54: Iteration calculation: 174
[ ] 2018-10-03 13:55:55: Iteration calculation: 175
[ ] 2018-10-03 13:55:55: Iteration calculation: 176
[ ] 2018-10-03 13:55:55: Iteration calculation: 177
[ ] 2018-10-03 13:55:55: Iteration calculation: 178
[ ] 2018-10-